In [1]:
from load_datas import Dataloader
from feature import get_features
# from dqn_worker import train_worker_dqn

dataloader = Dataloader()

train_data, valid_data, test_data = dataloader.get_datas()


train_r_data, train_w_data = get_features(train_data, print_size=True)
valid_r_data, valid_w_data = get_features(valid_data)
test_r_data, test_w_data = get_features(test_data)

initialized empty a_space_emb size:  torch.Size([1653, 8])


c:\Users\PyKing\Desktop\git_pull\RL-mid\feature.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  p_feature = torch.tensor(p_history[p_id2index[str(p_id)]], dtype=torch.float16)


final a_space_emb size:  torch.Size([137008, 1653, 8])
requester s1 size: torch.Size([137008, 1329])
requester s2 size: torch.Size([137008, 1329])
requester a size: torch.Size([137008])
initialized empty a_space_emb size:  torch.Size([1653, 8])
final a_space_emb size:  torch.Size([18079, 1653, 8])
initialized empty a_space_emb size:  torch.Size([1653, 8])
final a_space_emb size:  torch.Size([17390, 1653, 8])


In [2]:
import torch
torch.any(torch.isnan(train_r_data['a']))

tensor(False)

In [3]:
import torch

def de_dim(train_r_data, valid_r_data, test_r_data, key, dim=128):
    S = torch.cat([train_r_data[key], valid_r_data[key], test_r_data[key]], dim=0)
    
    n = S.size(0)
    S_centered = S - torch.mean(S, dim=0)

    # S_centered = S_centered.to(device)
    cov_matrix = torch.mm(S_centered.t(), S_centered) / (n - 1)
    print("Done cov")
    e_values, e_vectors = torch.linalg.eigh(cov_matrix)
    print("Done e")

    sorted_indices = torch.argsort(e_values, descending=True)
    e_vectors = e_vectors[:, sorted_indices]

    p_comp = e_vectors[:, :dim]
    S_reduced = torch.mm(S_centered, p_comp)
    print(S_reduced.size())
    torch.save(S_reduced, "data/pca_data/low_dim_%s.pt" % key)

de_dim(train_r_data, valid_r_data, test_r_data, 's1', dim=128)

Done cov
Done e
torch.Size([172477, 128])


In [4]:
de_dim(train_r_data, valid_r_data, test_r_data, 's2', dim=128)

Done cov
Done e
torch.Size([172477, 128])


In [5]:
key = 'a_space_emb'
S = torch.cat([train_r_data[key], valid_r_data[key], test_r_data[key]], dim=0)
torch.save(S, "data/pca_data/low_dim_%s.pt" % key)

In [6]:
key = 'a'
S = torch.cat([train_r_data[key], valid_r_data[key], test_r_data[key]], dim=0)
torch.save(S, "data/pca_data/low_dim_%s.pt" % key)

In [7]:
key = 'r'
S = torch.cat([torch.tensor(train_r_data[key]), torch.tensor(valid_r_data[key]), torch.tensor(test_r_data[key])], dim=0)
S = (S - S.min()) / S.max()
torch.save(S, "data/pca_data/low_dim_%s.pt" % key)

In [8]:
key = 'r'
S = torch.cat([torch.tensor(train_w_data[key]), torch.tensor(valid_w_data[key]), torch.tensor(test_w_data[key])], dim=0)
S = (S - S.min()) / S.max()
torch.save(S, "data/pca_data/low_dim_w_%s.pt" % key)

In [9]:
# 读数据代码：

import torch

train_num, valid_num = 137008, 18079
train_r_data, train_w_data, valid_r_data, valid_w_data, test_r_data, test_w_data = {}, {}, {}, {}, {}, {}

keys = ['s1', "s2", 'a', 'a_space_emb', 'r']
datas = {}
for key in keys:
    datas[key] = torch.load("data/pca_data/low_dim_%s.pt"%key)

for key in keys:
    train_r_data[key] = datas[key][:train_num]
    valid_r_data[key] = datas[key][train_num:train_num+valid_num]
    test_r_data[key] = datas[key][train_num+valid_num:]

wr = torch.load("data/pca_data/low_dim_w_r.pt")
train_w_data['r'] = wr[:train_num]
valid_w_data['r'] = wr[train_num:train_num+valid_num]
test_w_data['r'] = wr[train_num+valid_num:]


C:\Users\PyKing\AppData\Local\Temp\ipykernel_14152\470929991.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  datas[key] = torch.load("data/pca_data/low_dim_%s.pt"%key)


数据维度

In [10]:
print("train requester:")
for key in keys:
    print(key, train_r_data[key].size())
print("\nvalid requester:")
for key in keys:
    print(key, valid_r_data[key].size())
print("\ntest requester:")
for key in keys:
    print(key, test_r_data[key].size())

train requester:
s1 torch.Size([137008, 128])
s2 torch.Size([137008, 128])
a torch.Size([137008])
a_space_emb torch.Size([137008, 1653, 8])
r torch.Size([137008])

valid requester:
s1 torch.Size([18079, 128])
s2 torch.Size([18079, 128])
a torch.Size([18079])
a_space_emb torch.Size([18079, 1653, 8])
r torch.Size([18079])

test requester:
s1 torch.Size([17390, 128])
s2 torch.Size([17390, 128])
a torch.Size([17390])
a_space_emb torch.Size([17390, 1653, 8])
r torch.Size([17390])


In [11]:
print("train worker:")
print('r', train_w_data['r'].size())
print("valid worker:")
print('r', valid_w_data['r'].size())
print("test worker:")
print('r', test_w_data['r'].size())

train worker:
r torch.Size([137008])
valid worker:
r torch.Size([18079])
test worker:
r torch.Size([17390])
